# Importation des bibliothèques nécessaires
Importer les bibliothèques nécessaires, y compris pandas, numpy, scikit-learn, et matplotlib.

In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn style for plots
#sns.set(style="whitegrid")

import matplotlib.pyplot as plt
plt.style.use('ggplot')  # Appliquer le style ggplot


# Chargement des données
Charger les fichiers CSV HDFS_occurence_matrix_HDFS_train.csv, HDFS_occurence_matrix_HDFS_valid.csv, et HDFS_occurence_matrix_HDFS_test.csv dans des DataFrames pandas.

In [25]:
# Chargement des données
train_df = pd.read_csv('BGL_results/BGL_train_sliding_window.csv')
valid_df = pd.read_csv('BGL_results/BGL_valid_sliding_window.csv')
test_df = pd.read_csv('BGL_results/BGL_test_sliding_window.csv')

# Afficher les premières lignes des DataFrames pour vérifier le chargement
print(train_df.head())
print(valid_df.head())
print(test_df.head())

             Timestamp    E1  E2  E3  E4  E5  E6  E7  E8  E9  ...  E499  E500  \
0  2005-06-03 15:42:50  1625   1   0   0   0   0   0   0   0  ...     0     0   
1  2005-06-03 15:43:20  1640   1   0   0   0   0   0   0   0  ...     0     0   
2  2005-06-03 15:43:50  1637   1   0   0   0   0   0   0   0  ...     0     0   
3  2005-06-03 15:44:20  1632   1   0   0   0   0   0   0   0  ...     0     0   
4  2005-06-03 15:44:50  1593   1   0   0   0   0   0   0   0  ...     0     0   

   E501  E502  E503  E504  E505  E506  E507  Anomaly  
0     0     0     0     0     0     0     0     Fail  
1     0     0     0     0     0     0     0     Fail  
2     0     0     0     0     0     0     0     Fail  
3     0     0     0     0     0     0     0     Fail  
4     0     0     0     0     0     0     0     Fail  

[5 rows x 509 columns]
             Timestamp  E1  E2    E3  E4  E5  E6  E7  E8  E9  ...  E657  E658  \
0  2005-07-28 12:16:21   0   0  4879   0   0   0   0   0   0  ...     0     0 

# Analyse des variables et traitement des corrélations
Calculer la matrice de corrélation sur les variables indépendantes dans l'ensemble d'entraînement. Identifier et supprimer les variables fortement corrélées (corrélation > 0,9).

In [ ]:
# Suppression des colonnes non indépendantes et calcul de la matrice de corrélation
correlation_matrix = train_df.drop(columns=['Timestamp', 'Anomaly']).corr()

# Affichage de la matrice de corrélation avec des ajustements pour éviter le chevauchement
plt.figure(figsize=(14, 12))  # Augmenter la taille du graphique
sns.heatmap(
    correlation_matrix, 
    annot=True, 
    fmt=".2f", 
    cmap='coolwarm', 
    square=True, 
    annot_kws={"size": 8}  # Taille des annotations
)

# Ajuster l'alignement des étiquettes des axes
plt.xticks(rotation=45, ha='right', fontsize=10)  # Rotation des étiquettes sur l'axe x
plt.yticks(rotation=0, fontsize=10)  # Rotation des étiquettes sur l'axe y

plt.title('Matrice de corrélation des variables indépendantes', fontsize=14)
plt.tight_layout()  # Ajuste automatiquement les marges pour éviter les chevauchements
plt.show()

# Identification des variables fortement corrélées (corrélation > 0.9)
high_corr_var = set()
threshold = 0.9

# Parcourir les éléments de la matrice de corrélation
for i in range(len(correlation_matrix.columns)):
    for j in range(i):  # Éviter les doublons et la diagonale
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            high_corr_var.add(colname)

# Suppression des variables fortement corrélées
train_df_reduced = train_df.drop(columns=high_corr_var)
valid_df_reduced = valid_df.drop(columns=high_corr_var)
test_df_reduced = test_df.drop(columns=high_corr_var)

# Affichage des variables supprimées
print(f"Variables supprimées en raison d'une forte corrélation: {high_corr_var}")

# Vérification des DataFrames réduits
print("Aperçu des DataFrames réduits :")
print(train_df_reduced.head())
print(valid_df_reduced.head())
print(test_df_reduced.head())

# Création des échantillons avec bootstrap
Utiliser le bootstrap pour créer des échantillons de l'ensemble d'entraînement. Chaque échantillon a la même taille que l'ensemble d'entraînement d'origine (avec remplacement).

Pour chaque échantillon bootstrap, entraîner un modèle de régression logistique sur l'ensemble d'entraînement. Valider ce modèle sur un ensemble de validation distinct.

Répéter les étapes de création d'échantillons bootstrap et de modélisation un nombre défini de fois. Stocker les performances (Précision, Rappel et AUC).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# Définir le nombre d'échantillons bootstrap ou répétitions
n_iterations = 100

# Préparer les données
label_mapping = {'Fail': 1, 'Success': 0}
train_df_reduced['Anomaly'] = train_df_reduced['Anomaly'].map(label_mapping)
valid_df_reduced['Anomaly'] = valid_df_reduced['Anomaly'].map(label_mapping)

X_train = train_df_reduced.drop(columns=['Timestamp', 'Anomaly'])
y_train = train_df_reduced['Anomaly']
X_valid = valid_df_reduced.drop(columns=['Timestamp', 'Anomaly'])
y_valid = valid_df_reduced['Anomaly']

# Initialiser les listes pour stocker les performances des modèles
metrics_lr = {"accuracy": [], "precision": [], "recall": [], "f1": [], "roc_auc": []}
metrics_rf = {"accuracy": [], "precision": [], "recall": [], "f1": [], "roc_auc": []}

# Boucle principale pour le bootstrap et la modélisation
for _ in range(n_iterations):
    # Créer un échantillon bootstrap
    bootstrap_sample = resample(train_df_reduced, replace=True, n_samples=len(train_df_reduced), random_state=None)
    X_train_bootstrap = bootstrap_sample.drop(columns=['Timestamp', 'Anomaly'])
    y_train_bootstrap = bootstrap_sample['Anomaly']
    
    # Entraîner Logistic Regression
    model_lr = LogisticRegression(solver='saga', max_iter=5000, random_state=None)
    model_lr.fit(X_train_bootstrap, y_train_bootstrap)
    y_pred_lr = model_lr.predict(X_valid)
    y_pred_proba_lr = model_lr.predict_proba(X_valid)[:, 1]
    
    # Calculer les métriques pour Logistic Regression
    metrics_lr["accuracy"].append(accuracy_score(y_valid, y_pred_lr))
    metrics_lr["precision"].append(precision_score(y_valid, y_pred_lr))
    metrics_lr["recall"].append(recall_score(y_valid, y_pred_lr))
    metrics_lr["f1"].append(f1_score(y_valid, y_pred_lr))
    metrics_lr["roc_auc"].append(roc_auc_score(y_valid, y_pred_proba_lr))
    
    # Entraîner Random Forest
    model_rf = RandomForestClassifier(n_estimators=100, random_state=None)
    model_rf.fit(X_train_bootstrap, y_train_bootstrap)
    y_pred_rf = model_rf.predict(X_valid)
    y_pred_proba_rf = model_rf.predict_proba(X_valid)[:, 1]
    
    # Calculer les métriques pour Random Forest
    metrics_rf["accuracy"].append(accuracy_score(y_valid, y_pred_rf))
    metrics_rf["precision"].append(precision_score(y_valid, y_pred_rf))
    metrics_rf["recall"].append(recall_score(y_valid, y_pred_rf))
    metrics_rf["f1"].append(f1_score(y_valid, y_pred_rf))
    metrics_rf["roc_auc"].append(roc_auc_score(y_valid, y_pred_proba_rf))

# Afficher les performances moyennes pour Logistic Regression
print("\nPerformances moyennes pour Logistic Regression:")
for metric, values in metrics_lr.items():
    print(f"{metric.capitalize()} moyenne: {np.mean(values):.4f}")

# Afficher les performances moyennes pour Random Forest
print("\nPerformances moyennes pour Random Forest:")
for metric, values in metrics_rf.items():
    print(f"{metric.capitalize()} moyenne: {np.mean(values):.4f}")

# Visualisation des performances
plt.figure(figsize=(15, 10))

metrics_names = ["accuracy", "precision", "recall", "f1", "roc_auc"]
titles = ['Accuracy', 'Précision', 'Rappel', 'F1-Score', 'AUC-ROC']

for i, metric in enumerate(metrics_names):
    plt.subplot(2, 5, i + 1)
    sns.boxplot(data=metrics_lr[metric])
    plt.title(f'Logistic Regression: {titles[i]}')
    
    plt.subplot(2, 5, i + 6)
    sns.boxplot(data=metrics_rf[metric])
    plt.title(f'Random Forest: {titles[i]}')

plt.tight_layout()
plt.show()


# Évaluation finale sur le test
Appliquer le meilleur modèle sur l'ensemble de test. Calculer la matrice de confusion et des métriques comme l'accuracy, la précision, le rappel, le F1-Score, et l'AUC-ROC.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Combine training and validation sets
X_train_full = pd.concat([X_train, X_valid])
y_train_full = pd.concat([y_train, y_valid])

# Prepare test set features and target
X_test = test_df_reduced.drop(columns=['Timestamp', 'Anomaly'])
y_test = test_df_reduced['Anomaly']

# Convert y_test to numerical values
label_mapping = {'Fail': 1, 'Success': 0}
y_test = y_test.map(label_mapping)

# Handle missing values with imputation
imputer = SimpleImputer(strategy='mean')
X_train_full = pd.DataFrame(imputer.fit_transform(X_train_full), columns=X_train_full.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Verify the conversion
if y_test.isnull().any():
    raise ValueError("y_test contains NaN values after mapping. Check the data.")

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Store performance metrics
metrics = {}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"\n--- Modèle : {model_name} ---")
    # Train the model
    model.fit(X_train_full, y_train_full)

    # Predict on the test set
    y_test_pred = model.predict(X_test)
    y_test_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate performance metrics
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    f1_test = f1_score(y_test, y_test_pred)
    roc_auc_test = roc_auc_score(y_test, y_test_pred_proba)

    # Store metrics
    metrics[model_name] = {
        "Accuracy": accuracy_test,
        "Precision": precision_test,
        "Recall": recall_test,
        "F1-Score": f1_test,
        "AUC-ROC": roc_auc_test
    }

    # Display performance metrics
    print(f"Accuracy sur le test: {accuracy_test:.4f}")
    print(f"Précision sur le test: {precision_test:.4f}")
    print(f"Rappel sur le test: {recall_test:.4f}")
    print(f"F1-Score sur le test: {f1_test:.4f}")
    print(f"AUC-ROC sur le test: {roc_auc_test:.4f}")

    # Calculate and display the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues', xticklabels=['Success', 'Fail'], yticklabels=['Success', 'Fail'])
    plt.xlabel('Prédiction')
    plt.ylabel('Réalité')
    plt.title(f'Matrice de confusion - {model_name}')
    plt.show()

    # Plot the ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_test_pred_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC-ROC = {roc_auc_test:.4f}')
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    plt.xlabel('Taux de faux positifs (FPR)')
    plt.ylabel('Taux de vrais positifs (TPR)')
    plt.title(f'Courbe ROC - {model_name}')
    plt.legend(loc="lower right")
    plt.show()

# Compare the performance metrics of the two models
print("\n--- Comparaison des performances ---")
metrics_df = pd.DataFrame(metrics).T
print(metrics_df)

# Visualize the comparison as a bar chart
metrics_df.plot(kind='bar', figsize=(12, 6), rot=0)
plt.title("Comparaison des performances des modèles")
plt.ylabel("Valeurs des métriques")
plt.xlabel("Modèles")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


# Déterminer le meilleur modèle
Les performances des deux modèles sont affichées avec des valeurs pour chaque métrique.

Le modèle avec la meilleure performance globale est identifié et assigné à best_model.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Logistic Regression
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_full, y_train_full)
y_test_pred_lr = lr_model.predict(X_test)
y_test_pred_proba_lr = lr_model.predict_proba(X_test)[:, 1]

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_full, y_train_full)
y_test_pred_rf = rf_model.predict(X_test)
y_test_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Calculer les métriques pour Logistic Regression
lr_metrics = {
    "Accuracy": accuracy_score(y_test, y_test_pred_lr),
    "Precision": precision_score(y_test, y_test_pred_lr),
    "Recall": recall_score(y_test, y_test_pred_lr),
    "F1-Score": f1_score(y_test, y_test_pred_lr),
    "AUC-ROC": roc_auc_score(y_test, y_test_pred_proba_lr)
}

# Calculer les métriques pour Random Forest
rf_metrics = {
    "Accuracy": accuracy_score(y_test, y_test_pred_rf),
    "Precision": precision_score(y_test, y_test_pred_rf),
    "Recall": recall_score(y_test, y_test_pred_rf),
    "F1-Score": f1_score(y_test, y_test_pred_rf),
    "AUC-ROC": roc_auc_score(y_test, y_test_pred_proba_rf)
}

# Comparer les métriques
print("\nPerformances de Logistic Regression:")
for metric, value in lr_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nPerformances de Random Forest:")
for metric, value in rf_metrics.items():
    print(f"{metric}: {value:.4f}")

# Décider du modèle avec la meilleure performance globale (exemple basé sur AUC-ROC)
if lr_metrics["AUC-ROC"] > rf_metrics["AUC-ROC"]:
    print("\nLogistic Regression a performé le mieux selon AUC-ROC.")
    best_model = lr_model  # Assigner Logistic Regression comme meilleur modèle
else:
    print("\nRandom Forest a performé le mieux selon AUC-ROC.")
    best_model = rf_model  # Assigner Random Forest comme meilleur modèle


# Interprétation du modèle AiOps

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

# Extraire les coefficients pour Logistic Regression
coefficients = best_model.coef_[0]  # Coefficients du modèle Logistic Regression
feature_names = X_train_full.columns  # Noms des variables dans l'ensemble combiné

# Calculer l'importance des variables pour Logistic Regression
logistic_importances = pd.DataFrame({
    'Variable': feature_names,
    'Importance': np.abs(coefficients)  # Valeurs absolues des coefficients
}).sort_values(by='Importance', ascending=False)

# Entraîner un modèle Random Forest pour comparer les importances des variables
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_full, y_train_full)

# Calculer l'importance des variables pour Random Forest
rf_importances = pd.DataFrame({
    'Variable': feature_names,
    'Importance': rf_model.feature_importances_  # Importances données par Random Forest
}).sort_values(by='Importance', ascending=False)

# Afficher les importances pour Logistic Regression
print("\nImportance des variables - Régression Logistique :")
print(logistic_importances)

# Afficher les importances pour Random Forest
print("\nImportance des variables - Random Forest :")
print(rf_importances)

# Visualisation des importances pour Logistic Regression
plt.figure(figsize=(10, 6))
plt.barh(logistic_importances['Variable'], logistic_importances['Importance'], color='skyblue')
plt.xlabel('Importance (Magnitude des coefficients)')
plt.ylabel('Variables')
plt.title('Importance des Variables - Régression Logistique')
plt.gca().invert_yaxis()  # Inverser l'ordre pour afficher les plus importantes en haut
plt.show()

# Visualisation des importances pour Random Forest
plt.figure(figsize=(10, 6))
plt.barh(rf_importances['Variable'], rf_importances['Importance'], color='lightgreen')
plt.xlabel('Importance (Score d\'importance)')
plt.ylabel('Variables')
plt.title('Importance des Variables - Random Forest')
plt.gca().invert_yaxis()  # Inverser l'ordre pour afficher les plus importantes en haut
plt.show()

# Comparaison des importances entre les deux modèles
comparison = logistic_importances.merge(
    rf_importances, on='Variable', suffixes=('_LogisticRegression', '_RandomForest')
)

# Afficher la comparaison
print("\nComparaison des importances des variables :")
print(comparison)

# Visualisation de la comparaison
plt.figure(figsize=(12, 8))
comparison.plot(x='Variable', kind='bar', figsize=(12, 6))
plt.title("Comparaison des Importances des Variables entre Régression Logistique et Random Forest")
plt.ylabel("Importance")
plt.xlabel("Variables")
plt.legend(["Logistic Regression", "Random Forest"])
plt.tight_layout()
plt.show()
